# Ben is my best friend

### Load code libraries that we'll need

In [6]:
from bs4 import BeautifulSoup
import os
import pandas as pd
import requests

### Store all the URLs for all the states using this website format

In [2]:
URLS = {
  'AK': 'https://www.golfalaska.com/golfcourses/results.cfm?startrow=',    
  'AL': 'https://www.golfalabama.com/golfcourses/results.cfm?startrow=',
  'AZ': 'https://www.arizonagolfer.com/golfcourses/results.cfm?startrow=',
  'AR': 'https://www.arkansasgolf.com/golfcourses/results.cfm?startrow=',
  'CA': 'https://www.cagolf.com/golfcourses/results.cfm?startrow=',
  'CO': 'https://www.golfingcolorado.com/golfcourses/results.cfm?startrow=',
  'CT': 'https://www.connecticutgolf.com/golfcourses/results.cfm?startrow=',
  'DE': 'https://www.golfdelaware.com/golfcourses/results.cfm?startrow=',
  'DC': 'https://www.dcgolf.com/golfcourses/results.cfm?startrow=',
  'FL': 'https://www.floridagolfer.com/golfcourses/results.cfm?startrow=',
  'GA': 'https://www.golfinggeorgia.com/golfcourses/results.cfm?startrow=',
  'HI': 'https://www.hawaiigolfcourses.com/golfcourses/results.cfm?startrow=',
  'ID': 'https://www.idahogolf.com/golfcourses/results.cfm?startrow=',
  'IL': 'https://www.illinoisgolf.com/golfcourses/results.cfm?startrow=',
  'IN': 'https://www.indianagolf.com/golfcourses/results.cfm?startrow=',
  'IA': 'https://www.iowagolf.com/golfcourses/results.cfm?startrow=',
  'KS': 'https://www.kansasgolf.com/golfcourses/results.cfm?startrow=',
  'KY': 'https://www.kentuckygolf.com/golfcourses/results.cfm?startrow=',
  'LA': 'https://www.louisianagolf.com/golfcourses/results.cfm?startrow=',
  'ME': 'https://www.mainegolf.com/golfcourses/results.cfm?startrow=',
  'MD': 'https://www.marylandgolf.com/golfcourses/results.cfm?startrow=',
  'MA': 'https://www.golfmassachusetts.com/golfcourses/results.cfm?startrow=',
  'MI': 'https://www.golfmichigan.com/golfcourses/results.cfm?startrow=',
  'MN': 'https://www.minnesotagolf.com/golfcourses/results.cfm?startrow=',
  'MS': 'https://www.golfmississippi.com/golfcourses/results.cfm?startrow=',
  'MO': 'https://www.golfmissouri.com/golfcourses/results.cfm?startrow=',
  'MT': 'https://www.golfingmontana.com/golfcourses/results.cfm?startrow=',
  'NE': 'https://www.golfnebraska.com/golfcourses/results.cfm?startrow=',
  'NV': 'https://www.golfnevada.com/golfcourses/results.cfm?startrow=',
  'NH': 'https://www.newhampshiregolf.com/golfcourses/results.cfm?startrow=',
  'NJ': 'https://www.njgolf.com/golfcourses/results.cfm?startrow=',
  'NM': 'https://www.golfingnewmexico.com/golfcourses/results.cfm?startrow=',
  'NY': 'https://www.nygolf.com/golfcourses/results.cfm?startrow=',
  'NC': 'https://www.ncgolf.com/golfcourses/results.cfm?startrow=',
  'ND': 'https://www.northdakotagolf.com/golfcourses/results.cfm?startrow=',
  'OH': 'https://www.ohiogolf.com/golfcourses/results.cfm?startrow=',
  'OK': 'https://www.golfoklahoma.com/golfcourses/results.cfm?startrow=',
  'OR': 'https://www.golforegon.com/golfcourses/results.cfm?startrow=',
  'PA': 'https://www.pennsylvaniagolfer.com/golfcourses/results.cfm?startrow=',
  'RI': 'https://www.rhodeislandgolf.com/golfcourses/results.cfm?startrow=',
  'SC': 'https://www.scgolf.com/golfcourses/results.cfm?startrow=',
  'SD': 'https://www.southdakotagolf.com/golfcourses/results.cfm?startrow=',
  'TN': 'https://www.golftennessee.com/golfcourses/results.cfm?startrow=',
  'TX': 'https://www.texasgolf.com/golfcourses/results.cfm?startrow=',
  'UT': 'https://www.utahgolf.com/golfcourses/results.cfm?startrow=',
  'VT': 'https://www.golfvermont.com/golfcourses/results.cfm?startrow=',
  'VA': 'https://www.golfvirginia.com/golfcourses/results.cfm?startrow=',
  'WA': 'https://www.washingtongolf.com/golfcourses/results.cfm?startrow=',
  'WV': 'https://www.golfwestvirginia.com/golfcourses/results.cfm?startrow=',
  'WI': 'https://www.golfwisconsin.com/golfcourses/results.cfm?startrow=',
  'WY': 'https://www.golfwyoming.com/golfcourses/results.cfm?startrow=',
}

OUTPUT_FILENAME = 'all_courses.csv'

### Helper functions

In [29]:
# Given a direct url to a single course's information page, find and return the phone number.
def get_course_phone(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text)
    elt = soup.find_all('h4')[0]  # The first h4 is the one we want
    lines = elt.text.splitlines()
    for line in lines:
        line = line.strip()  # Remove whitespace
        if line.startswith('Phone:'):
            # Will look like: 'Phone: 270-646-4653'
            return line.split()[1]  # Return everything after the first space

In [62]:
# Return data for (up to) 20 golf courses on the given website, starting with course number "row_start"
def get_page(base_url, row_start, data):
    url = base_url + str(row_start)
    response = requests.get(url)

    # From 'https://www.kentuckygolf.com/golfcourses/results.cfm?startrow='
    # extract the substring, 'https://www.kentuckygolf.com/golfcourses/'
    # We'll use this to fetch individual course phone numbers later.
    root_url = url.split('results.cfm')[0]

    # Find all the golf course information blocks, which are identified by
    # an <h4> tag with an <a> nested inside it (which has the course name
    # and a link to its website). We are looking for things like:
    # <h4><a href="../golfcourses/barren-river-lake-golf-course">Barren River Lake State Resort Park</a></h4>
    soup = BeautifulSoup(response.text)
    headers = soup.find_all('h4')
    filtered_headers = []
    for elt in headers:
        if elt.find('a'):
            filtered_headers.append(elt)

    results = []
    if not filtered_headers:
        return results, False
    all_fetched_before = True
    for elt in filtered_headers:
        url = elt.find('a').attrs['href']  # Will look like '../golfcourses/barren-river-lake-golf-course'
        url = root_url + url.split('/golfcourses/')[1]  # Extract 'barren-river-lake-golf-course'
        if url in data['url'].values:
            continue
        all_fetched_before = False
        print(len(results) + row_start, end=' ')
        name = elt.text.strip()  # Remove whitespace
        address = elt.findNext('p').findNext('p').text  # Address is listed two <p> tags later
        address = ' '.join(address.split())  # Remove interstitial white space
        phone = get_course_phone(url)  # Load the course page and find the phone number
        results.append([name, address, phone, url])
    print()
    return results, all_fetched_before

### Fetch each state's data, writing each to its own file

In [63]:
def extract_zip(address):
    # 3651 OMalley Rd Anchorage, AK 99516
    if isinstance(address, str):
        zipcode = address.split()[-1]
        if zipcode.isdigit():
            return int(zipcode)
        return None
    return None

In [65]:
if os.path.exists(OUTPUT_FILENAME):
    data = pd.read_csv(OUTPUT_FILENAME)
else:
    data = pd.DataFrame([], columns=['name', 'address', 'state', 'zip', 'phone', 'url'])
# data = pd.DataFrame([], columns=['name', 'address', 'state', 'zip', 'phone', 'url'])

# Loop through each state and URL pair
for state, url in URLS.items():
    print(state)
    row = 1
    while True:
        print("Fetching 20 courses starting with number", row)
        new_results, all_fetched_before = get_page(url, row, data)
        
        # Create a DataFrame from all the results
        new_data = pd.DataFrame(new_results, columns=['name', 'address', 'phone', 'url'])
        new_data['state'] = state
        new_data['zip'] = new_data['address'].apply(extract_zip)
        
        data = pd.concat([data, new_data])
        
        # Save the DataFrame to a single CSV file
        data.to_csv(OUTPUT_FILENAME, index=False)
    
        if (len(new_results) < 20) and not all_fetched_before:
            break
        row += 20
    print(sum(data['state'] == state), 'pulled for', state)
    print()
